In [33]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import sklearn.datasets as sd
import sklearn.linear_model
np.random.seed(1)

In [15]:
X,Y = sd.load_iris(return_X_y=True)

In [16]:
X = X.reshape(4,150)
Y = Y.reshape(1,150)
X.shape,Y.shape

((4, 150), (1, 150))

In [17]:
# training examples
m = Y.shape[1]
m

150

In [18]:
def layer_sizes(X, Y):
    """
    Arguments:
    X -- input dataset of shape (input size, number of examples)
    Y -- labels of shape (output size, number of examples)
    
    Returns:
    n_x -- the size of the input layer
    n_h -- the size of the hidden layer
    n_y -- the size of the output layer
    """
    
    n_x =  X.shape[0]# size of input layer
    n_h1=  4
    n_h2=  3
    n_y =  Y.shape[0]# size of output layer
    return (n_x, n_h1,n_h2, n_y)

In [19]:
def initialize_parameters(n_x, n_h1,n_h2, n_y):
    """
    Argument:
    n_x -- size of the input layer
    n_h -- size of the hidden layer
    n_y -- size of the output layer
    
    Returns:
    params -- python dictionary containing your parameters:
                    W1 -- weight matrix of shape (n_h, n_x)
                    b1 -- bias vector of shape (n_h, 1)
                    W2 -- weight matrix of shape (n_y, n_h)
                    b2 -- bias vector of shape (n_y, 1)
    """
    
    np.random.seed(2) # we set up a seed so that your output matches ours although the initialization is random.
    
    W1 = np.random.randn(n_h1,n_x)*0.01
    b1 = np.zeros((n_h1,1))
    W2 = np.random.randn(n_h2,n_h1)*0.01
    b2 =np.zeros((n_h2,1))
    W3 = np.random.randn(n_y,n_h2)*0.01
    b3 =np.zeros((n_y,1))
   
    parameters = {"W1": W1,"b1": b1,
                  "W2": W2,"b2": b2,
                  "W3":W3,"b3":b3}
    
    return parameters

In [20]:
def sigmoid(x):
    """
    Compute the sigmoid of x

    Arguments:
    x -- A scalar or numpy array of any size.

    Return:
    s -- sigmoid(x)
    """
    s = 1/(1+np.exp(-x))
    return s

In [21]:
def forward_propagation(X, parameters):
    """
    Argument:
    X -- input data of size (n_x, m)
    parameters -- python dictionary containing your parameters (output of initialization function)
    
    Returns:
    A2 -- The sigmoid output of the second activation
    cache -- a dictionary containing "Z1", "A1", "Z2" and "A2"
    """
    # Retrieve each parameter from the dictionary "parameters"
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    W3 = parameters["W3"]
    b3 = parameters["b3"]
    # Implement Forward Propagation to calculate A2 (probabilities)
    Z1 = np.dot(W1,X)+b1
    A1 = np.tanh(Z1)
    Z2 = np.dot(W2,A1)+b2
    A2 = sigmoid(Z2)
    Z3 = np.dot(W3,A2)+b3
    A3 = sigmoid(Z3)

    cache = {"Z1": Z1,
             "A1": A1,
             "Z2": Z2,
             "A2": A2,
             "Z3": Z3,
             "A3": A3}
    
    return A3, cache

In [22]:
def compute_cost(A3, Y, parameters):
    """
    Computes the cross-entropy cost given in equation (13)
    
    Arguments:
    A2 -- The sigmoid output of the second activation, of shape (1, number of examples)
    Y -- "true" labels vector of shape (1, number of examples)
    parameters -- python dictionary containing your parameters W1, b1, W2 and b2
    
    Returns:
    cost -- cross-entropy cost given equation (13)
    """
    m = Y.shape[1] 
    # Compute the cross-entropy cost
    logprobs = np.multiply(np.log(A3),Y)+np.multiply(1-Y,np.log(1-A3))
    
    cost = -np.sum(logprobs)/m
    
    cost = np.squeeze(cost)     
    assert(isinstance(cost, float))
    return cost

In [36]:
def backward_propagation(parameters, cache, X, Y):
    """
    Implement the backward propagation using the instructions above.
    
    Arguments:
    parameters -- python dictionary containing our parameters 
    cache -- a dictionary containing "Z1", "A1", "Z2" and "A2".
    X -- input data of shape (2, number of examples)
    Y -- "true" labels vector of shape (1, number of examples)
    
    Returns:
    grads -- python dictionary containing your gradients with respect to different parameters
    """
   
    m = X.shape[1]

    W1 = parameters["W1"]
    W2 = parameters["W2"]
    W3 = parameters["W3"]    
    A1 = cache["A1"]
    A2 = cache["A2"]
    A3 = cache["A3"]
    
    dW3=(1/m)*np.dot(A3-Y,A2.T)
    db3 =(1/m)*np.sum(A3-Y,axis=1,keepdims=True)
    sub=np.multiply(np.dot(W3.T,A3-Y),1-np.power(A2,2))
    dW2=(1/m)*np.dot(sub,A1.T)
    db2=(1/m)*np.sum(sub,axis=1,keepdims=True)
    sub1=np.multiply(np.dot(W2.T,sub),1-np.power(A1,2))
    dW1=(1/m)*np.dot(sub1,X.T)
    db1=(1/m)*np.sum(sub1,axis=1,keepdims=True)
    
    grads = {"dW1": dW1,"db1": db1,
             "dW2": dW2,"db2": db2,
             "dW3": dW3,"db3": db3}
    
    return grads

In [44]:
def update_parameters(parameters, grads, learning_rate = 0.005):
    """
    Updates parameters using the gradient descent update rule given above
    
    Arguments:
    parameters -- python dictionary containing your parameters 
    grads -- python dictionary containing your gradients 
    
    Returns:
    parameters -- python dictionary containing your updated parameters 
    """
   
    W1 = parameters["W1"]
    b1 = parameters["b1"] 
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    W3 = parameters["W3"]
    b3 = parameters["b3"]
    alpha = learning_rate
    # Retrieve each gradient from the dictionary "grads"
   
    dW1 = grads["dW1"]
    db1 = grads["db1"]
    dW2 = grads["dW2"]
    db2 = grads["db2"]
    dW3 = grads["dW3"]
    db3 = grads["db3"]
    
    # Update rule for each parameter
    
    W1 = W1 - alpha *dW1
    b1 = b1 - alpha *db1
    W2 = W2 - alpha *dW2
    b2 = b2 - alpha *db2
    W3 = W3 - alpha *dW3
    b3 = b3 - alpha *db3
    
    parameters = {"W1": W1,"b1": b1,
                  "W2": W2,"b2": b2,
                  "W3":W3,"b3":b3}
    
    return parameters

In [45]:
def nn_model(X, Y, n_h1,n_h2, num_iterations = 100000, print_cost=False):
    """
    Arguments:
    X -- dataset of shape (2, number of examples)
    Y -- labels of shape (1, number of examples)
    n_h -- size of the hidden layer
    num_iterations -- Number of iterations in gradient descent loop
    print_cost -- if True, print the cost every 1000 iterations
    
    Returns:
    parameters -- parameters learnt by the model. They can then be used to predict.
    """
    
    np.random.seed(3)
    
    n_x = layer_sizes(X, Y)[0]
    n_y = layer_sizes(X, Y)[3]
    parameters = initialize_parameters(n_x,n_h1,n_h2,n_y)
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    W3 = parameters["W3"]
    b3 = parameters["b3"]
    # Loop (gradient descent)

    for i in range(0, num_iterations):
        # Forward propagation. Inputs: "X, parameters". Outputs: "A2, cache".
        A3, cache = forward_propagation(X, parameters)        
        # Cost function. Inputs: "A2, Y, parameters". Outputs: "cost".
        cost = compute_cost(A3,Y,parameters)
 
        # Backpropagation. Inputs: "parameters, cache, X, Y". Outputs: "grads".
        grads = backward_propagation(parameters,cache,X,Y)
 
        # Gradient descent parameter update. Inputs: "parameters, grads". Outputs: "parameters".
        parameters = update_parameters(parameters,grads)
        
        
        # Print the cost every 1000 iterations
        if print_cost and i % 10 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))

    return parameters

In [46]:

def predict(parameters, X):
    """
    Using the learned parameters, predicts a class for each example in X
    
    Arguments:
    parameters -- python dictionary containing your parameters 
    X -- input data of size (n_x, m)
    
    Returns
    predictions -- vector of predictions of our model (red: 0 / blue: 1)
    """
    
    # Computes probabilities using forward propagation, and classifies to 0/1 using 0.5 as the threshold.
    A3, cache = forward_propagation(X,parameters)
    predictions = np.round(A3)
    
    return predictions,A3

In [47]:
X1=X[0:5,0:30]
X2=X[0:5,50:80]

train_X=np.hstack((X1,X2))
test_X=np.hstack((X[0:5,30:50],X[0:5,80:100]))
Y1=Y[0:1,0:30]
Y2=Y[0:1,50:80]

train_Y=np.hstack((Y1,Y2))
test_Y=np.hstack((Y[0:1,30:50],Y[0:1,80:100]))
train_X.shape,train_Y.shape,test_X.shape,test_Y.shape

((4, 60), (1, 60), (4, 40), (1, 40))

In [48]:
parameters = nn_model(train_X, train_Y, n_h1 = 4,n_h2= 3, num_iterations = 10000, print_cost=True)

Cost after iteration 0: 0.693158
Cost after iteration 10: 0.693157
Cost after iteration 20: 0.693157
Cost after iteration 30: 0.693156
Cost after iteration 40: 0.693156
Cost after iteration 50: 0.693156
Cost after iteration 60: 0.693155
Cost after iteration 70: 0.693155
Cost after iteration 80: 0.693155
Cost after iteration 90: 0.693154
Cost after iteration 100: 0.693154
Cost after iteration 110: 0.693154
Cost after iteration 120: 0.693153
Cost after iteration 130: 0.693153
Cost after iteration 140: 0.693153
Cost after iteration 150: 0.693153
Cost after iteration 160: 0.693152
Cost after iteration 170: 0.693152
Cost after iteration 180: 0.693152
Cost after iteration 190: 0.693152
Cost after iteration 200: 0.693152
Cost after iteration 210: 0.693151
Cost after iteration 220: 0.693151
Cost after iteration 230: 0.693151
Cost after iteration 240: 0.693151
Cost after iteration 250: 0.693151
Cost after iteration 260: 0.693151
Cost after iteration 270: 0.693150
Cost after iteration 280: 0.693

Cost after iteration 2500: 0.693147
Cost after iteration 2510: 0.693147
Cost after iteration 2520: 0.693147
Cost after iteration 2530: 0.693147
Cost after iteration 2540: 0.693147
Cost after iteration 2550: 0.693147
Cost after iteration 2560: 0.693147
Cost after iteration 2570: 0.693147
Cost after iteration 2580: 0.693147
Cost after iteration 2590: 0.693147
Cost after iteration 2600: 0.693147
Cost after iteration 2610: 0.693147
Cost after iteration 2620: 0.693147
Cost after iteration 2630: 0.693147
Cost after iteration 2640: 0.693147
Cost after iteration 2650: 0.693147
Cost after iteration 2660: 0.693147
Cost after iteration 2670: 0.693147
Cost after iteration 2680: 0.693147
Cost after iteration 2690: 0.693147
Cost after iteration 2700: 0.693147
Cost after iteration 2710: 0.693147
Cost after iteration 2720: 0.693147
Cost after iteration 2730: 0.693147
Cost after iteration 2740: 0.693147
Cost after iteration 2750: 0.693147
Cost after iteration 2760: 0.693147
Cost after iteration 2770: 0

Cost after iteration 4970: 0.693147
Cost after iteration 4980: 0.693147
Cost after iteration 4990: 0.693147
Cost after iteration 5000: 0.693147
Cost after iteration 5010: 0.693147
Cost after iteration 5020: 0.693147
Cost after iteration 5030: 0.693147
Cost after iteration 5040: 0.693147
Cost after iteration 5050: 0.693147
Cost after iteration 5060: 0.693147
Cost after iteration 5070: 0.693147
Cost after iteration 5080: 0.693147
Cost after iteration 5090: 0.693147
Cost after iteration 5100: 0.693147
Cost after iteration 5110: 0.693147
Cost after iteration 5120: 0.693147
Cost after iteration 5130: 0.693147
Cost after iteration 5140: 0.693147
Cost after iteration 5150: 0.693147
Cost after iteration 5160: 0.693147
Cost after iteration 5170: 0.693147
Cost after iteration 5180: 0.693147
Cost after iteration 5190: 0.693147
Cost after iteration 5200: 0.693147
Cost after iteration 5210: 0.693147
Cost after iteration 5220: 0.693147
Cost after iteration 5230: 0.693147
Cost after iteration 5240: 0

Cost after iteration 7460: 0.693147
Cost after iteration 7470: 0.693147
Cost after iteration 7480: 0.693147
Cost after iteration 7490: 0.693147
Cost after iteration 7500: 0.693147
Cost after iteration 7510: 0.693147
Cost after iteration 7520: 0.693147
Cost after iteration 7530: 0.693147
Cost after iteration 7540: 0.693147
Cost after iteration 7550: 0.693147
Cost after iteration 7560: 0.693147
Cost after iteration 7570: 0.693147
Cost after iteration 7580: 0.693147
Cost after iteration 7590: 0.693147
Cost after iteration 7600: 0.693147
Cost after iteration 7610: 0.693147
Cost after iteration 7620: 0.693147
Cost after iteration 7630: 0.693147
Cost after iteration 7640: 0.693147
Cost after iteration 7650: 0.693147
Cost after iteration 7660: 0.693147
Cost after iteration 7670: 0.693147
Cost after iteration 7680: 0.693147
Cost after iteration 7690: 0.693147
Cost after iteration 7700: 0.693147
Cost after iteration 7710: 0.693147
Cost after iteration 7720: 0.693147
Cost after iteration 7730: 0

In [49]:
predictions,A3 = predict(parameters, test_X)
print ('Accuracy: %d' % float((np.dot(test_Y,predictions.T) + np.dot(1-test_Y,1-predictions.T))/float(test_Y.size)*100) + '%')

Accuracy: 50%
